In [1]:
!pip install dmba

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 17.9 MB/s eta 0:00:00


In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 44.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [3]:
#import required modules
from zipfile import ZipFile
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction import _stop_words
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from nltk import word_tokenize
from nltk import punkt
from nltk.stem.snowball import EnglishStemmer
import matplotlib.pylab as plt
from dmba import printTermDocumentMatrix, classificationSummary, liftChart
from datasets import load_dataset

Colab environment detected.


In [4]:
#import required modules
from zipfile import ZipFile
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from transformers import BertTokenizer, TFAutoModel, BertForSequenceClassification

In [46]:
import os
import shutil

In [5]:
newsWire = load_dataset('ag_news')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [6]:
newsWire.data

{'train': MemoryMappedTable
 text: string
 label: int64
 ----
 text: [["Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.","Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.","Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.","Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.","Oil prices soar to

In [7]:
corpus = pd.DataFrame(columns=['text', 'label'])

In [8]:
corpus['text'] = newsWire['train']['text']
corpus['label'] = newsWire['train']['label']

In [9]:
test_corpus = pd.DataFrame(columns=['text', 'label'])

In [10]:

test_corpus['text'] = newsWire['test']['text']
test_corpus['label'] = newsWire['test']['label']

In [11]:
corpus.head(20)

,text,label
0,Wall St. Bears Claw Back Into the Black (Reute...,2
1,Carlyle Looks Toward Commercial Aerospace (Reu...,2
2,Oil and Economy Cloud Stocks' Outlook (Reuters...,2
3,Iraq Halts Oil Exports from Main Southern Pipe...,2
4,"Oil prices soar to all-time record, posing new...",2
5,"Stocks End Up, But Near Year Lows (Reuters) Re...",2
6,Money Funds Fell in Latest Week (AP) AP - Asse...,2
7,Fed minutes show dissent over inflation (USATO...,2
8,Safety Net (Forbes.com) Forbes.com - After ear...,2
9,Wall St. Bears Claw Back Into the Black NEW Y...,2


In [12]:
test_corpus.head(15)

,text,label
0,Fears for T N pension after talks Unions repre...,2
1,The Race is On: Second Private Team Sets Launc...,3
2,Ky. Company Wins Grant to Study Peptides (AP) ...,3
3,Prediction Unit Helps Forecast Wildfires (AP) ...,3
4,Calif. Aims to Limit Farm-Related Smog (AP) AP...,3
5,Open Letter Against British Copyright Indoctri...,3
6,"Loosing the War on Terrorism \\""Sven Jaschan, ...",3
7,"FOAFKey: FOAF, PGP, Key Distribution, and Bloo...",3
8,E-mail scam targets police chief Wiltshire Pol...,3
9,"Card fraud unit nets 36,000 cards In its first...",3


In [13]:
corpus['label'].value_counts()

label
2    30000
3    30000
1    30000
0    30000
Name: count, dtype: int64

## Pre Process both training and validation data to be used accordingly in tensorflow

In [14]:
sequence_len = 512

train_samples = len(corpus)
test_samples = len(test_corpus)

train_ids_x = np.zeros((train_samples, sequence_len))
train_masks_x = np.zeros((train_samples, sequence_len))

test_ids_x = np.zeros((test_samples, sequence_len))
test_masks_x = np.zeros((test_samples, sequence_len))

In [17]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [18]:
for i, text in enumerate(corpus['text']):
    tokens = bert_tokenizer.encode_plus(text, max_length=sequence_len, truncation=True, padding='max_length', add_special_tokens=True, return_tensors='tf')
    train_ids_x[i, :] = tokens['input_ids']
    train_masks_x[i, :] = tokens['attention_mask']

for i, text in enumerate(test_corpus['text']):
    tokens = bert_tokenizer.encode_plus(text, max_length=sequence_len, truncation=True, padding='max_length', add_special_tokens=True, return_tensors='tf')
    test_ids_x[i, :] = tokens['input_ids']
    test_masks_x[i, :] = tokens['attention_mask']

In [19]:
train_labels_array = corpus['label'].values
test_labels_array = test_corpus['label'].values

In [20]:
np.unique(train_labels_array)

array([0, 1, 2, 3])

In [21]:
np.unique(test_labels_array)

array([0, 1, 2, 3])

In [22]:
#Encoding the samples for the text using one-hot encoding
train_labels = np.zeros((train_samples, len(np.unique(train_labels_array))))
train_labels[np.arange(train_samples), train_labels_array-1] = 1

test_labels = np.zeros((test_samples, len(np.unique(test_labels_array))))
test_labels[np.arange(test_samples), test_labels_array-1] = 1

In [23]:
test_labels.shape

(7600, 4)

In [24]:
#create a tensorflow dataset using the data
train_data = tf.data.Dataset.from_tensor_slices((train_ids_x, train_masks_x, train_labels))
test_data = tf.data.Dataset.from_tensor_slices((test_ids_x, test_masks_x, test_labels))

In [25]:
train_data

<_TensorSliceDataset element_spec=(TensorSpec(shape=(512,), dtype=tf.float64, name=None), TensorSpec(shape=(512,), dtype=tf.float64, name=None), TensorSpec(shape=(4,), dtype=tf.float64, name=None))>

In [26]:
#change the shape of the data to be accepted by tf as input and labels
def mapper(ids, masks, labels):
  return {'ids': ids, 'masks':masks }, labels

train_data = train_data.map(mapper)
test_data = test_data.map(mapper)

In [27]:
train_data

<_MapDataset element_spec=({'ids': TensorSpec(shape=(512,), dtype=tf.float64, name=None), 'masks': TensorSpec(shape=(512,), dtype=tf.float64, name=None)}, TensorSpec(shape=(4,), dtype=tf.float64, name=None))>

In [28]:
#create batches out of all the data for batch training
batch_size = 32
train_data = train_data.batch(batch_size, drop_remainder=True)
test_data = test_data.batch(batch_size, drop_remainder=True)

In [29]:
train_data.take(1)

<_TakeDataset element_spec=({'ids': TensorSpec(shape=(32, 512), dtype=tf.float64, name=None), 'masks': TensorSpec(shape=(32, 512), dtype=tf.float64, name=None)}, TensorSpec(shape=(32, 4), dtype=tf.float64, name=None))>

Build the models

In [30]:
BERT = TFAutoModel.from_pretrained('bert-base-uncased')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [31]:
BERT.summary()

Model: "tf_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
Total params: 109482240 (417.64 MB)
Trainable params: 109482240 (417.64 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
BERT2 = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(np.unique(train_labels_array)),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Create the input layers

In [33]:
inputs = tf.keras.layers.Input(shape=(sequence_len,), name='ids', dtype='int32')
attention_masks = tf.keras.layers.Input(shape=(sequence_len,), name='masks', dtype='int32')

In [34]:
#Use the transformer to get the embeddings for the inputs
#getting the 2d tensors
embeddings = BERT.bert(inputs, attention_mask=attention_masks)[1]

In [35]:
embeddings

<KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'bert')>

In [36]:
#Set up the dense layer and the
dense_layer_1 = tf.keras.layers.Dense(1024, activation = 'relu')(embeddings)
output_layer = tf.keras.layers.Dense(len(np.unique(train_labels_array)), activation = 'softmax', name='output_layer')(dense_layer_1)

In [37]:
model = tf.keras.Model(inputs=[inputs, attention_masks], outputs = output_layer)

In [38]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 ids (InputLayer)            [(None, 512)]                0         []                            
                                                                                                  
 masks (InputLayer)          [(None, 512)]                0         []                            
                                                                                                  
 bert (TFBertMainLayer)      TFBaseModelOutputWithPooli   1094822   ['ids[0][0]',                 
                             ngAndCrossAttentions(last_   40         'masks[0][0]']               
                             hidden_state=(None, 512, 7                                           
                             68),                                                             

In [39]:
#Optimizer to reduce the cross entropy loss
#Setting up the metrics
adam_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

In [40]:
model.compile(optimizer=adam_optimizer, loss=loss, metrics=[acc])

In [41]:
log = model.fit(train_data,validation_data = test_data, epochs = 3)

Epoch 1/3
3750/3750 [==============================] - 3102s 811ms/step - loss: 0.2403 - accuracy: 0.9181 - val_loss: 0.2055 - val_accuracy: 0.9353
Epoch 2/3
3750/3750 [==============================] - 3034s 809ms/step - loss: 0.1445 - accuracy: 0.9506 - val_loss: 0.1816 - val_accuracy: 0.9429
Epoch 3/3
3750/3750 [==============================] - 3034s 809ms/step - loss: 0.1048 - accuracy: 0.9645 - val_loss: 0.1958 - val_accuracy: 0.9419


In [53]:
model.save('/content/drive/MyDrive/BERT_Classifier')

In [54]:
loaded_model = tf.keras.models.load_model('/content/drive/MyDrive/BERT_Classifier')

In [55]:
loaded_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 ids (InputLayer)            [(None, 512)]                0         []                            
                                                                                                  
 masks (InputLayer)          [(None, 512)]                0         []                            
                                                                                                  
 bert (TFBertMainLayer)      TFBaseModelOutputWithPooli   1094822   ['ids[0][0]',                 
                             ngAndCrossAttentions(last_   40         'masks[0][0]']               
                             hidden_state=(None, 512, 7                                           
                             68),                                                             

##Save Model to drive

In [45]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [49]:
!cat /content/BERT_AG_news_classifier_609

cat: /content/BERT_AG_news_classifier_609: Is a directory


In [52]:
!cp -a /content/BERT_AG_news_classifier_609/. /content/drive/MyDrive/BERT_Classifier